### File to begin basic work on building the provenance graph and event distributions.

In [1]:
%run ./read_file.ipynb 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/26 19:40:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/26 19:40:36 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [30]:
df_eps = readCheckpoint('ecar', 'prod', 'small')

--- 2.9945085048675537 seconds ---


In [16]:
df = readCheckpoint('ecar', 'dev', 'small')

--- 0.9164233207702637 seconds ---


In [4]:
df.printSchema()

root
 |-- action: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- object: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- pid: integer (nullable = true)
 |-- ppid: integer (nullable = true)
 |-- principal: string (nullable = true)
 |-- tid: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- acuity_level: string (nullable = true)
 |-- base_address: string (nullable = true)
 |-- command_line: string (nullable = true)
 |-- context_info: string (nullable = true)
 |-- data: string (nullable = true)
 |-- dest_port: integer (nullable = true)
 |-- direction: string (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- file_path: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- info_class: string (nullable = true)
 |-- key: string (nullable = true)
 |-- l4protocol: string (nullable = true)
 |-- logon_id: string (nullable 

In [45]:
df.select("timestamp").limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,timestamp
0,2019-09-23 13:11:02.370
1,2019-09-23 13:11:02.375
2,2019-09-23 13:11:02.385
3,2019-09-23 13:11:02.409
4,2019-09-23 13:11:02.414


In [46]:
df.select(["objectID", "actorID","timestamp", "object","action"]).limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,objectID,actorID,timestamp,object,action
0,eeee9e56-df6e-476c-bfc7-03d507dbd8e7,c183f6f4-e83e-4d05-98df-6cafce2ec010,2019-09-23 13:11:02.370,FLOW,INFO
1,c7bfc171-627d-4103-b883-d5c3d4fd2f4d,c183f6f4-e83e-4d05-98df-6cafce2ec010,2019-09-23 13:11:02.375,FLOW,INFO
2,da018e32-281d-4a74-83cc-c113fc70900b,c183f6f4-e83e-4d05-98df-6cafce2ec010,2019-09-23 13:11:02.385,FLOW,INFO
3,b48bef19-32d5-4727-9a29-4e0186dcffe0,c183f6f4-e83e-4d05-98df-6cafce2ec010,2019-09-23 13:11:02.409,FLOW,INFO
4,cd7be00d-3691-484c-aedb-954582d6353c,c183f6f4-e83e-4d05-98df-6cafce2ec010,2019-09-23 13:11:02.414,FLOW,INFO


In [47]:
#what objects are represented
df.select(col("object")).distinct().count()

1

In [49]:
#how many event hours are represented
df.select(col("event_hour")).distinct().count()

7

In [6]:
#create column of month days to determine which days are represented. 
df = df.withColumn("day", dayofmonth(col("timestamp")))
df.select(col("day")).distinct().show()

+---+
|day|
+---+
| 23|
+---+



#### so we see only that the 23 is represented. 

In [17]:
#what is the size of the dataset? 
df.count()

1000000

In [23]:
#make a very small data set and see how many distinct actorIDs there are
df_vsmall = df.limit(10000)
df_vsmall.select(col("actorID")).distinct().count()

68

In [28]:
#great now order v_small dataset by time
df_vsmall.sort(col("timestamp").desc()).select("timestamp").limit(2).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,timestamp
0,2019-09-23 13:12:56.806
1,2019-09-23 13:12:56.806


In [29]:
#sort by ascending to see difference
df_vsmall.sort(col("timestamp").asc()).select("timestamp").limit(2).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,timestamp
0,2019-09-23 13:12:01.622
1,2019-09-23 13:12:01.622


In [31]:
df_vsmall.select("timestamp").limit(2).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,timestamp
0,2019-09-23 13:12:01.622
1,2019-09-23 13:12:01.723


In [24]:
#k so I have to sort the df and keep it that way. 
df_vsmall = df_vsmall.sort(col("timestamp").asc())

In [25]:
#check order
df_vsmall.select(["id","timestamp"]).limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp
0,76f6091b-412f-41eb-bcd3-9184aeac62a1,2019-09-23 13:11:00.145
1,17af483c-22ca-4710-b26c-f35924056575,2019-09-23 13:11:00.164
2,e6651435-1708-4c87-9478-f84ee9f5569c,2019-09-23 13:11:00.168
3,7309e9fd-9315-4200-b2d7-5a2ae29d92d4,2019-09-23 13:11:00.168
4,336c4edd-85b3-4694-909a-325b55b256e3,2019-09-23 13:11:00.171


In [36]:
#extract list of unique actor ids 
actIDs = [row.actorID for row in df_vsmall.select("actorID").distinct().collect()]
print(actIDs)

['89f91b70-9613-4e70-9473-23bb029e3889', 'e518426c-0868-49c2-a424-f22273343f80', '099b52e3-caf5-487d-bb04-5d5380049562', 'aa240ddc-17cd-4f29-bb39-e40a51f088b2', '095b125e-695e-456d-b962-7e3a131ff689', 'b81fd4f9-3e18-4260-9145-c14f68146fd9', 'a42bf7c9-bc3b-435a-891a-f1de9d8590aa']


In [26]:
#create an array of unique values for speed and then creat proof of concept iteration
test_ID_arr = df_vsmall.select("actorID").distinct().collect()
#for row in test_ID_arr: 
 #   print(row.actorID)

In [12]:
#confirm that event IDs are all unique. This should return a count of 50. 
df_vsmall.select("id").distinct().count()

1000

In [13]:
test_ID_arr[0].actorID

'c183f6f4-e83e-4d05-98df-6cafce2ec010'

In [27]:
#objID collection
test_objID = df_vsmall.select("objectID").distinct().collect()

In [15]:
#so there are nine examples of the same objID in the dataset
df_vsmall.where(col("objectID") == test_objID[0].objectID).count()

1

In [33]:
#but the obj ID appears 10 times in the actor ID...I think it's starting to make sense. 
df_vsmall.where(col("actorID") == test_objID[0].objectID).count()

0

In [28]:
#are there more objIDs or actorIDs
print("Objects: " + str(len(test_objID)))
print("Actors: " + str(len(test_ID_arr)))

Objects: 10000
Actors: 68


In [29]:
df_vsmall.select("objectID").distinct().count()

10000

In [30]:
test_ID_arr[1].actorID

'2b8a82ef-89df-4e07-b6a3-00c9e904d348'

In [32]:
#k this is obvious now, i was seeing the first actor 0 times in objID because it was the first event
df_vsmall.select(["objectID", "actorID","timestamp", "object","action"]).where(col("objectID") == test_ID_arr[10].actorID).toPandas()

,objectID,actorID,timestamp,object,action


In [20]:
#k this is obvious now, i was seeing the first actor 0 times in objID because it was the first event
df_vsmall.select(["objectID", "actorID","timestamp", "object","action"]).where(col("objectID") == test_ID_arr[2].actorID).toPandas()

,objectID,actorID,timestamp,object,action


In [51]:
for i in range(len(test_ID_arr)):
    if df_vsmall.where(col("objectID") == test_ID_arr[i].actorID).count() > 0: 
        print("yes")


In [18]:
#alright so the truncated data isn't working let's expand to 2 hours worth of data and see how many actor ids are in the
#obj ID collection. 

#objID collection
test_objID_big = df.select("objectID").distinct().collect()
test_ID_arr_big = df.select("actorID").distinct().collect()

print("Objects: " + str(len(test_objID_big)))
print("Actors: " + str(len(test_ID_arr_big)))


Objects: 48622
Actors: 2327


In [19]:
#what events are in the "full dataset"
df.select("object").distinct().show()

+-------+
| object|
+-------+
|  SHELL|
|   FLOW|
|   FILE|
|PROCESS|
+-------+



In [31]:
df_eps.select("object").distinct().show()

+------+
|object|
+------+
|  FLOW|
+------+



In [15]:
#proof of concept

samp = [{"actorIDs" : ['a','b','c'], \
               "objectIDs" : ['c','d']}]
id_intersect = spark.createDataFrame(samp)
id_intersect = (id_intersect.withColumn("id_int", \
                                        array_intersect(col("actorIDs"), col("objectIDs"))))
id_intersect.show()

+---------+---------+------+
| actorIDs|objectIDs|id_int|
+---------+---------+------+
|[a, b, c]|   [c, d]|   [c]|
+---------+---------+------+



In [20]:
#so I used the wrong code to find the overlap between the two "sets" of ids. instead
#I need to create an array with one row and two columns. the columns will contain lists of Obj and
#actor IDs. I will create a spark df with that array. I will then create another column with array interesct, which
#will give me a list of the common elements. I can then use the overlapping elements to build traces.

data = [{"actorIDs" : [row.actorID for row in test_ID_arr_big], \
               "objectIDs" : [row.objectID for row in test_objID_big]}]
id_intersect = spark.createDataFrame(data)


In [21]:
#what is the overlap
id_intersect = (id_intersect.withColumn("id_int", \
                                        array_intersect(col("actorIDs"), col("objectIDs"))))

In [25]:
id_intersect.select("id_int").toPandas()

,id_int
0,"[86d3fc1e-dfe9-4482-a888-527838b43f77, 49b0ed0..."


In [29]:
#extract the overlap list and turn it into a column
overlap = id_intersect.select("id_int").collect()
overlap = [row.id_int for row in overlap][0]
print(len(overlap))

1715


#### We show 1715 actor ids that are in the object IDs! huzzah! I probably have to sort and then run this all again.

In [33]:
df.printSchema()

root
 |-- action: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- object: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- pid: integer (nullable = true)
 |-- ppid: integer (nullable = true)
 |-- principal: string (nullable = true)
 |-- tid: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- acuity_level: string (nullable = true)
 |-- base_address: string (nullable = true)
 |-- command_line: string (nullable = true)
 |-- context_info: string (nullable = true)
 |-- data: string (nullable = true)
 |-- dest_port: integer (nullable = true)
 |-- direction: string (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- file_path: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- info_class: string (nullable = true)
 |-- key: string (nullable = true)
 |-- l4protocol: string (nullable = true)
 |-- logon_id: string (nullable 

In [ ]:
#sort the column by timestamp, re-run intersect, then find the first actor ID
#in the object ID column, and not the actor ID column, and add that to a dictionary as a key, or a node. 
